In [1]:
from keras.applications import MobileNet
img_rows,img_cols=224,224

Using TensorFlow backend.
C:\Users\Arifiya khan\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Arifiya khan\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Arifiya khan\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Arifiya khan\anaconda3\lib\site-packages\tensorflo

In [2]:
model=MobileNet(weights='imagenet',include_top=False,input_shape=(224,224,3))

In [3]:
model.layers[0].input

<tf.Tensor 'input_1:0' shape=(?, 224, 224, 3) dtype=float32>

In [4]:
model.layers

In [5]:
model.layers[0].__class__.__name__

'InputLayer'

In [6]:
model.layers[2].trainable

True

In [7]:
for layer in modelt.layers:
    layer.trainable=False

NameError: name 'modelt' is not defined

In [8]:
for (i,layer) in enumerate(model.layers) :
    print(str(i) + " " + layer.__class__.__name__,layer.trainable)

0 InputLayer False
1 ZeroPadding2D True
2 Conv2D True
3 BatchNormalization True
4 ReLU True
5 DepthwiseConv2D True
6 BatchNormalization True
7 ReLU True
8 Conv2D True
9 BatchNormalization True
10 ReLU True
11 ZeroPadding2D True
12 DepthwiseConv2D True
13 BatchNormalization True
14 ReLU True
15 Conv2D True
16 BatchNormalization True
17 ReLU True
18 DepthwiseConv2D True
19 BatchNormalization True
20 ReLU True
21 Conv2D True
22 BatchNormalization True
23 ReLU True
24 ZeroPadding2D True
25 DepthwiseConv2D True
26 BatchNormalization True
27 ReLU True
28 Conv2D True
29 BatchNormalization True
30 ReLU True
31 DepthwiseConv2D True
32 BatchNormalization True
33 ReLU True
34 Conv2D True
35 BatchNormalization True
36 ReLU True
37 ZeroPadding2D True
38 DepthwiseConv2D True
39 BatchNormalization True
40 ReLU True
41 Conv2D True
42 BatchNormalization True
43 ReLU True
44 DepthwiseConv2D True
45 BatchNormalization True
46 ReLU True
47 Conv2D True
48 BatchNormalization True
49 ReLU True
50 DepthwiseCo

In [9]:
def lw(bottom_model,num_classes):

    top_model=bottom_model.output
    top_model=GlobalAveragePooling2D()(top_model)
    top_model=Dense(1024,activation='relu')(top_model)
    top_model=Dense(800,activation='relu')(top_model)
    top_model=Dense(600,activation='relu')(top_model)
    top_model=Dense(400,activation='relu')(top_model)
    top_model=Dense(300,activation='relu')(top_model)
    top_model=Dense(200,activation='relu')(top_model)
    top_model=Dense(128,activation='relu')(top_model)
    top_model=Dense(64,activation='relu')(top_model)
    top_model=Dense(10,activation='relu')(top_model)
    top_model=Dense(num_classes,activation='softmax')(top_model)
    return top_model
                                    

In [10]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,GlobalAveragePooling2D
from keras.layers import Conv2D,MaxPooling2D,ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

In [12]:
num_classes=2
FC_Head= lw(model,num_classes)
model=Model(inputs=model.input,outputs=FC_Head)
print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [13]:
model.input

<tf.Tensor 'input_1:0' shape=(?, 224, 224, 3) dtype=float32>

In [14]:
from keras.preprocessing.image import ImageDataGenerator
train_data_dir='dataset/train_data/'
validation_data_dir='dataset/test_data/'

In [15]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                rotation_range=45,
                                width_shift_range=0.3,
                                height_shift_range=0.3,
                                horizontal_flip=True,
                                vertical_flip=True,
                                fill_mode='nearest')

validation_datagen=ImageDataGenerator(rescale=1./255)
batch_size=1

train_generator=train_datagen.flow_from_directory(train_data_dir,
                                                  target_size=(img_rows,img_cols),
                                                  batch_size=batch_size,
                                                  class_mode='categorical')
validation_generator=validation_datagen.flow_from_directory(validation_data_dir,
                                                  target_size=(img_rows,img_cols),
                                                  batch_size=batch_size,
                                                  class_mode='categorical')
                               
                               

Found 1126 images belonging to 2 classes.
Found 191 images belonging to 2 classes.


In [16]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint,EarlyStopping

In [17]:
checkpoint=ModelCheckpoint("face_recog.h5",
                          monitor="val_loss",
                          mode="min",
                          save_best_only=True,
                          verbose=1)

earlystop=EarlyStopping(monitor="val_loss",
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True)

callbacks=[earlystop,checkpoint]

model.compile(loss='binary_crossentropy',
             optimizer=RMSprop(lr=0.0001),
             metrics= ['accuracy'])

nb_train_samples=800
nb_validation_sample=150

epochs=10
batch_size=16

history=model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples //batch_size,
        epochs=epochs,
        callbacks=callbacks,
        validation_data=validation_generator,
        validation_steps=nb_validation_sample // batch_size)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/10
50/50 [==============================] - 37s 738ms/step - loss: 0.6600 - accuracy: 0.7200 - val_loss: 0.0957 - val_accuracy: 0.4444

Epoch 00001: val_loss improved from inf to 0.09574, saving model to face_recog.h5
Epoch 2/10
50/50 [==============================] - 32s 637ms/step - loss: 0.7381 - accuracy: 0.6200 - val_loss: 1.1239 - val_accuracy: 0.4444

Epoch 00002: val_loss did not improve from 0.09574
Epoch 3/10
50/50 [==============================] - 32s 641ms/step - loss: 0.7148 - accuracy: 0.6000 - val_loss: 0.5379 - val_accuracy: 0.4444

Epoch 00003: val_loss did not improve from 0.09574
Epoch 4/10
50/50 [==============================] - 32s 645ms/step - loss: 0.7118 - accuracy: 0.4800 - val_loss: 0.7324 - val_accuracy: 0.3333
Restoring model weights from the end of the best epoch

Epoch 00004: val_loss did not improve from 0.09574
Epoch 00004: early stopping


In [18]:
from keras.models import load_model

In [19]:
classifier=load_model('face_recog.h5')

In [24]:
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile,join

face_model_dict={
                  "[0]":"arifiya",
                  "[1]":"Bhai"
                }

face_model_dict_1={
                  "arifiya":"arifiya",
                  "Bhai":"Bhai"
                }
def draw_test(name,pred,im):
    face=face_model_dict[str(pred)]
    BLACK=[0,0,0]
    expanded_image=cv2.copyMakeBorder(im,80,0,0,100,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image,face,(20,60),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
    cv2.imshow(name,expanded_image)

In [ ]:
def getRandomImage(path):
    folders=list(filter(lambda x: os.path.isdir(os.path.join(path,x)),os.listdir(path)))
    random_directory=np.random.randint(0,len(folders))
    path_class=folders[random_directory]
    print("class - " + face_model_dict_1[str(path_class)])
    file_path=path+path_class
    file_names=[f for f in listdir(file_path) if isfile(join(file_path,f))]
    random_file_index=np.random.randint(0,len(file_names))
    image_name=file_names[random_file_index]
    return cv2.imread(file_path + "/" + image_name)

for i in range (0,20):
    input_im=getRandomImage("dataset/test_data/")
    input_original=input_im.copy()
    input_original=cv2.resize(input_original,None,fx=0.5,fy=0.5,interpolation=cv2.INTER_LINEAR)
    input_im=cv2.resize(input_im,(224,224),interpolation=cv2.INTER_LINEAR)
    input_im=input_im / 225.
    input_im=input_im.reshape(1,224,224,3)
    
    
    res=np.argmax(classifier.predict(input_im,1,verbose=0),axis=1)
    draw_test("Prediction",res,input_original)
    cv2.waitKey(0)
    
cv2.destroyAllWindows()


class - Bhai
class - arifiya
class - Bhai
class - arifiya
class - Bhai
class - arifiya
class - Bhai
class - Bhai
